# tt̄ spin observables: stacks + P,C,D (using explicit sumWeights)

In [1]:
import os, math, numpy as np, sys, csv
from pathlib import Path
import matplotlib.pyplot as plt
import mplhep as hep
USE_ROOT = True
try:
    import ROOT
    ROOT.gErrorIgnoreLevel = ROOT.kError
    ROOT.TH1.SetDefaultSumw2(True)
except Exception as e:
    USE_ROOT = False
    import uproot
out_dir = 'compare_plots_mpl_stack'
os.makedirs(out_dir, exist_ok=True)
print('Backend:', 'PyROOT' if USE_ROOT else 'uproot')


Backend: PyROOT


In [2]:
data_files = [
    'DoubleMuon_Data_2018.root',
    'MuonEG_Data_2018.root',
    'EGamma_Data_2018.root',
    'SingleMuon_Data_2018.root',
]
mc_files = [
    'DYJetsToLL_M-10to50_MC_2018.root',
    'DYJetsToLL_M-50_MC_2018.root',
    'ST_s-channel_MC_2018.root',
    'ST_t-channel_antitop_MC_2018.root',
    'ST_t-channel_top_MC_2018.root',
    'ST_tW_antitop_MC_2018.root',
    'ST_tW_top_MC_2018.root',
    'TTTo2L2Nu_MC_2018.root',
    'TTToHadronic_MC_2018.root',
    'TTToSemiLeptonic_MC_2018.root',
    'WJetsToLNu_MC_2018.root',
    'WW_MC_2018.root',
    'WZ_MC_2018.root',
    'ZZ_MC_2018.root',
]
cross_sections_pb = {
    'DYJetsToLL_M-10to50_MC_2018.root': 18610.0,
    'DYJetsToLL_M-50_MC_2018.root':    6077.22,
    'ST_s-channel_MC_2018.root':       3.36,
    'ST_t-channel_antitop_MC_2018.root': 80.95,
    'ST_t-channel_top_MC_2018.root':   136.02,
    'ST_tW_antitop_MC_2018.root':      34.91,
    'ST_tW_top_MC_2018.root':          34.97,
    'TTTo2L2Nu_MC_2018.root':          87.4,
    'TTToHadronic_MC_2018.root':       380.0,
    'TTToSemiLeptonic_MC_2018.root':   364.0,
    'WJetsToLNu_MC_2018.root':         61526.0,
    'WW_MC_2018.root':                 118.7,
    'WZ_MC_2018.root':                 47.13,
    'ZZ_MC_2018.root':                 16.523,
}
sum_weights = {
    'DYJetsToLL_M-50_MC_2018.root':        131572454.00,
    'TTToSemiLeptonic_MC_2018.root':       472557630.00,
    'DYJetsToLL_M-10to50_MC_2018.root':    74373706.00,
    'ZZ_MC_2018.root':                      3526000.00,
    'TTToHadronic_MC_2018.root':          331506194.00,
    'ST_t-channel_antitop_MC_2018.root':   90022642.00,
    'ST_s-channel_MC_2018.root':           12607741.00,
    'TTTo2L2Nu_MC_2018.root':             143848848.00,
    'ST_tW_top_MC_2018.root':               7955614.00,
    'WJetsToLNu_MC_2018.root':             79645994.00,
    'ST_tW_antitop_MC_2018.root':           7748690.00,
    'ST_t-channel_top_MC_2018.root':      167111718.00,
    'WW_MC_2018.root':                     15679000.00,
    'WZ_MC_2018.root':                      7940000.00,
}
luminosity_pb = 59.740 * 1000.0
alpha_l = 1.0


In [3]:
def get_hist_names(filename, directory='plots'):
    if USE_ROOT:
        f = ROOT.TFile.Open(filename)
        if not f or f.IsZombie(): return []
        if not f.GetDirectory(directory): f.Close(); return []
        f.cd(directory)
        names = [k.GetName() for k in ROOT.gDirectory.GetListOfKeys()]
        f.Close(); return names
    else:
        with uproot.open(filename) as f:
            if directory not in f: return []
            return [k.split(';')[0] for k in f[directory].keys()]

hist_names = get_hist_names(mc_files[0])
prefixes = ('mumu','ee','emu','combine','numberof','ll','all')
hist_names = [h for h in hist_names if any(h.startswith(p) for p in prefixes)]
print('Found', len(hist_names), 'histograms')


Found 257 histograms


In [4]:
def read_hist(file_name, hist_path):
    if USE_ROOT:
        f = ROOT.TFile.Open(file_name)
        if not f or f.IsZombie(): return None
        h = f.Get(hist_path)
        if h:
            h = h.Clone(f'tmp_{os.path.basename(file_name)}_{h.GetName()}')
            h.SetDirectory(0)
        f.Close(); return h
    else:
        with uproot.open(file_name) as f:
            if hist_path not in f: return (None, None)
            h = f[hist_path]
            try:
                edges = h.axis().edges(); counts = h.values()
            except Exception:
                counts, edges = h.to_numpy()
            return (np.asarray(edges), np.asarray(counts))

def hist_to_numpy_root(h):
    nb = h.GetNbinsX()
    edges = np.array([h.GetBinLowEdge(i) for i in range(1, nb+1)] + [h.GetBinLowEdge(nb)+h.GetBinWidth(nb)])
    counts = np.array([h.GetBinContent(i) for i in range(1, nb+1)])
    return edges, counts


In [5]:
def get_scale_factor(mc_filename):
    xsec = cross_sections_pb.get(mc_filename, 1.0)
    n_evt = sum_weights.get(mc_filename, None)
    if n_evt is None or n_evt <= 0:
        print(f'[warn] sumWeights missing or non-positive for {mc_filename}; skipping this sample.')
        return 0.0
    return (xsec * luminosity_pb) / n_evt


In [6]:
def stack_mc_numpy(hname):
    hist_path = f'plots/{hname}'
    arrays, labels = [], []
    edges_common = None
    mc_sum_hist = None
    for fn in mc_files:
        scale = get_scale_factor(fn)
        if scale <= 0: continue
        if USE_ROOT:
            h = read_hist(fn, hist_path)
            if not h: continue
            h.Sumw2(); h.Scale(scale)
            edges, counts = hist_to_numpy_root(h)
            if edges_common is None: edges_common = edges
            arrays.append(counts); labels.append(os.path.splitext(fn)[0])
            if mc_sum_hist is None:
                mc_sum_hist = h.Clone('mc_sum'); mc_sum_hist.SetDirectory(0)
            else:
                mc_sum_hist.Add(h)
        else:
            edges, counts = read_hist(fn, hist_path)
            if edges is None: continue
            counts = counts * scale
            if edges_common is None: edges_common = edges
            arrays.append(counts); labels.append(os.path.splitext(fn)[0])
    if edges_common is None or len(arrays)==0:
        return None, [], [], None, None
    arrays = [np.asarray(a) for a in arrays]
    mc_sum_counts = np.sum(arrays, axis=0) if len(arrays)>0 else None
    return edges_common, arrays, labels, mc_sum_counts, mc_sum_hist

def combine_data(hname):
    hist_path = f'plots/{hname}'
    if USE_ROOT:
        data_hist = None
        for fn in data_files:
            h = read_hist(fn, hist_path)
            if not h: continue
            if data_hist is None:
                data_hist = h.Clone('data_sum'); data_hist.SetDirectory(0)
            else:
                data_hist.Add(h)
        if data_hist is None: return (None, None)
        return hist_to_numpy_root(data_hist)
    else:
        edges, counts = None, None
        for fn in data_files:
            e, c = read_hist(fn, hist_path)
            if e is None: continue
            if edges is None:
                edges, counts = e, c.copy()
            else:
                if not np.allclose(edges, e):
                    raise RuntimeError('Inconsistent data binning')
                counts += c
        return edges, counts


In [7]:
def is_cos_hist(hname):
    keys = [
        'cosThetaPlus_k','cosThetaMinus_k','cosThetaPlus_k_times_Minus_k',
        'cosThetaPlus_r','cosThetaMinus_r','cosThetaPlus_r_times_Minus_r',
        'cosThetaPlus_n','cosThetaMinus_n','cosThetaPlus_n_times_Minus_n',
    ]
    return any(k in hname for k in keys)
def axis_from_name(hname):
    if '_k' in hname: return 'k'
    if '_r' in hname: return 'r'
    if '_n' in hname: return 'n'
    return None
def channel_from_name(hname):
    for ch in ['ee','mumu','emu','all']:
        if hname.startswith(ch+'_'): return ch
    return 'all'
def mean_from_counts(edges, counts):
    centers = 0.5*(edges[:-1]+edges[1:])
    tot = counts.sum()
    if tot <= 0: return float('nan')
    return float(np.dot(centers, counts)/tot)


In [8]:
plt.style.use(hep.style.CMS)
results = []
for hname in hist_names:
    edges, dat_counts = combine_data(hname)
    if edges is None:
        print('[skip] No data for', hname); continue
    dat_err = np.sqrt(np.clip(dat_counts,0,None))
    centers = 0.5*(edges[:-1]+edges[1:])
    edges_mc, mc_arrays, mc_labels, mc_sum_counts, mc_sum_hist = stack_mc_numpy(hname)
    if edges_mc is None:
        print('[skip] No MC for', hname); continue
    cmap = plt.get_cmap('tab20')
    others = [lbl for lbl in mc_labels if lbl != 'TTTo2L2Nu_MC_2018']
    color_map = {lbl: cmap(i/max(1,len(others)-1)) for i,lbl in enumerate(others)}
    color_map['TTTo2L2Nu_MC_2018'] = 'red'
    mc_colors = [color_map[lbl] for lbl in mc_labels]
    fig, (ax, axr) = plt.subplots(nrows=2, sharex=True, gridspec_kw={'height_ratios':[3,1],'hspace':0.05}, figsize=(10,8), dpi=150)
    hep.histplot(mc_arrays, bins=edges, stack=True, histtype='fill', label=mc_labels, color=mc_colors, ax=ax)
    ax.errorbar(centers, dat_counts, yerr=dat_err, fmt='o', color='black', label='Data')
    ax.set_ylabel('Events / bin')
    hep.cms.label('Private Work', data=True, lumi=59.8, ax=ax)
    ax.legend(loc='upper right', prop={'size':8}, handletextpad=0.2, labelspacing=0.2, columnspacing=0.5)
    mc_sum = mc_sum_counts
    mask = mc_sum > 0
    ratio = np.divide(dat_counts, mc_sum, out=np.zeros_like(dat_counts), where=mask)
    ratio_err = np.divide(dat_err, mc_sum, out=np.zeros_like(dat_err), where=mask)
    axr.errorbar(centers[mask], ratio[mask], yerr=ratio_err[mask], fmt='o', color='black')
    axr.axhline(1, color='gray', linestyle='--', linewidth=1)
    axr.set_ylabel('Data/MC'); axr.set_xlabel(hname); axr.set_ylim(0.5,1.5)
    fig.savefig(f"{out_dir}/stack_{hname}.png"); plt.close(fig)
    if is_cos_hist(hname):
        ch = channel_from_name(hname)
        axis = axis_from_name(hname)
        mean_data = mean_from_counts(edges, dat_counts)
        if USE_ROOT and mc_sum_hist is not None:
            mean_mc = float(mc_sum_hist.GetMean())
        else:
            mean_mc = mean_from_counts(edges, mc_sum_counts)
        if 'times_Minus_' in hname:
            C_data = (9.0/(alpha_l**2)) * mean_data
            C_mc   = (9.0/(alpha_l**2)) * mean_mc
            results.append([hname, ch, axis, 'Caa', mean_data, mean_mc, C_data, C_mc])
        elif 'cosThetaPlus_' in hname:
            P_data = (3.0/alpha_l) * mean_data
            P_mc   = (3.0/alpha_l) * mean_mc
            results.append([hname, ch, axis, 'Pa', mean_data, mean_mc, P_data, P_mc])
import pandas as pd
C_map_data = {ch:{'k':None,'r':None,'n':None} for ch in ['ee','mumu','emu','all']}
C_map_mc   = {ch:{'k':None,'r':None,'n':None} for ch in ['ee','mumu','emu','all']}
for hname,ch,axis,kind,mean_d,mean_m,val_d,val_m in results:
    if kind=='Caa':
        C_map_data[ch][axis]=val_d; C_map_mc[ch][axis]=val_m
for ch in C_map_data:
    cds=C_map_data[ch]; cms=C_map_mc[ch]
    if all(v is not None for v in cds.values()):
        D_data=(cds['k']+cds['r']+cds['n'])/3.0
        results.append([f'{ch}_D',ch,'sum','D',float('nan'),float('nan'),D_data,float('nan')])
    if all(v is not None for v in cms.values()):
        D_mc=(cms['k']+cms['r']+cms['n'])/3.0
        results.append([f'{ch}_D_mc',ch,'sum','D_MC',float('nan'),float('nan'),float('nan'),D_mc])
df = pd.DataFrame(results, columns=['hname','channel','axis','kind','mean_data','mean_mc','value_data','value_mc'])
csv_path = f"{out_dir}/spin_observables.csv"; df.to_csv(csv_path, index=False)
print('Saved:', csv_path)


Saved: compare_plots_mpl_stack/spin_observables.csv
